# Continuous Training with Kubeflow Pipeline and Vertex AI

**Learning Objectives:**
1. Learn how to use KF pre-built components
1. Learn how to build a KF pipeline with these components
1. Learn how to compile, upload, and run a KF pipeline


In this lab, you will build, deploy, and run a KFP pipeline that orchestrates the **Vertex AI** services to train, tune, and deploy a **scikit-learn** model using the Google pre-built components.

## Setup

In [90]:
import os
from datetime import datetime

from google.cloud import aiplatform
aiplatform.init(location="us-east1")

In [91]:
REGION = "us-east1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]

In [92]:
# Set `PATH` to include the directory containing KFP CLI
PATH = %env PATH
%env PATH=/home/jupyter/.local/bin:{PATH}

env: PATH=/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/home/jupyter/.local/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games


## Understanding the pipeline design

The workflow implemented by the pipeline is defined using a Python based Domain Specific Language (DSL). The pipeline's DSL is in the `pipeline_vertex/pipeline_prebuilt.py` file that we will generate below.

The pipeline's DSL has been designed to avoid hardcoding any environment specific settings like file paths or connection strings. These settings are provided to the pipeline code through a set of environment variables.

### Build the trainer image

In [93]:
TRAINING_APP_FOLDER = "training_app"
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)

In [94]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune implicit scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Overwriting training_app/Dockerfile


Let's now build and push this trainer container to the container registry:

In [95]:
IMAGE_NAME = "trainer_image_beer_vertex"
TAG = "latest"
TRAINING_CONTAINER_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}"
TRAINING_CONTAINER_IMAGE_URI

'gcr.io/qwiklabs-asl-04-5e165f533cac/trainer_image_beer_vertex:latest'

In [96]:
!gcloud builds submit --timeout 15m --tag $TRAINING_CONTAINER_IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 4 file(s) totalling 16.8 KiB before compression.
Uploading tarball of [training_app] to [gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654832564.930777-922a08ad22c146739c583129872c0653.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-04-5e165f533cac/locations/global/builds/9f7360a3-e6d1-4dfd-9d75-bb034239bb8e].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/9f7360a3-e6d1-4dfd-9d75-bb034239bb8e?project=547029906128].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "9f7360a3-e6d1-4dfd-9d75-bb034239bb8e"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654832564.930777-922a08ad22c146739c583129872c0653.tgz#1654832565199753
Copying gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654832564.930777-922a08ad22c146739c583129872c0653.tgz#1654832565199753...
/ [1 files][  3.3 KiB/  3.3 KiB]                   

In [97]:
SERVING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest"
)

## Building and deploying the pipeline

Let us write the pipeline to disk:

In [98]:
%%writefile ./pipeline/pipeline_prebuilt.py
# Copyright 2021 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License"); you may not
# use this file except in compliance with the License. You may obtain a copy of
# the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
# express or implied. See the License for the specific language governing
# permissions and limitations under the License.
"""Kubeflow Covertype Pipeline."""
import os

from google.cloud.aiplatform import hyperparameter_tuning as hpt
from google_cloud_pipeline_components.aiplatform import (
    EndpointCreateOp,
    ModelDeployOp,
    ModelUploadOp,
)
from google_cloud_pipeline_components.experimental import (
    hyperparameter_tuning_job,
)
from google_cloud_pipeline_components.experimental.custom_job import (
    CustomTrainingJobOp,
)
from kfp.v2 import dsl

PIPELINE_ROOT = os.getenv("PIPELINE_ROOT")
PROJECT_ID = os.getenv("PROJECT_ID")
REGION = os.getenv("REGION")

TRAINING_CONTAINER_IMAGE_URI = os.getenv("TRAINING_CONTAINER_IMAGE_URI")
SERVING_CONTAINER_IMAGE_URI = os.getenv("SERVING_CONTAINER_IMAGE_URI")
SERVING_MACHINE_TYPE = os.getenv("SERVING_MACHINE_TYPE", "n1-standard-16")

TRAINING_FILE_PATH = os.getenv("TRAINING_FILE_PATH")
VALIDATION_FILE_PATH = os.getenv("VALIDATION_FILE_PATH")

MAX_TRIAL_COUNT = int(os.getenv("MAX_TRIAL_COUNT", "5"))
PARALLEL_TRIAL_COUNT = int(os.getenv("PARALLEL_TRIAL_COUNT", "5"))
THRESHOLD = float(os.getenv("THRESHOLD", "0.6"))

PIPELINE_NAME = os.getenv("PIPELINE_NAME", "beer")
BASE_OUTPUT_DIR = os.getenv("BASE_OUTPUT_DIR", PIPELINE_ROOT)
MODEL_DISPLAY_NAME = os.getenv("MODEL_DISPLAY_NAME", PIPELINE_NAME)

@dsl.pipeline(
    name=f"{PIPELINE_NAME}-kfp-pipeline",
    description="Kubeflow pipeline that tunes, trains, and deploys on Vertex",
    pipeline_root=PIPELINE_ROOT,
)
def create_pipeline():

    worker_pool_specs = [
        {
            "machine_spec": {
                "machine_type": "n1-standard-16",
            },
            "replica_count": 1,
            "container_spec": {
                "image_uri": TRAINING_CONTAINER_IMAGE_URI,
            },
        }
    ]

    metric_spec = hyperparameter_tuning_job.serialize_metrics(
        {"map_at_10": "maximize"}
    )

    parameter_spec = hyperparameter_tuning_job.serialize_parameters(
        {
            "regularization": hpt.DoubleParameterSpec(
                min=1.0e-4, max=1.0e-1, scale="log"
            ),
            "factors": hpt.DiscreteParameterSpec(
                values=[16, 32, 64], scale="linear"
            ),
            "iterations": hpt.IntegerParameterSpec(
                min=10, max=100, scale="linear"
            ),
        }
    )

    hp_tuning_task = hyperparameter_tuning_job.HyperparameterTuningJobRunOp(
        display_name=f"{PIPELINE_NAME}-kfp-tuning-job",
        project=PROJECT_ID,
        location=REGION,
        worker_pool_specs=worker_pool_specs,
        study_spec_metrics=metric_spec,
        study_spec_parameters=parameter_spec,
        max_trial_count=MAX_TRIAL_COUNT,
        parallel_trial_count=PARALLEL_TRIAL_COUNT,
        base_output_directory=PIPELINE_ROOT,
    )

    trials_task = hyperparameter_tuning_job.GetTrialsOp(
        gcp_resources=hp_tuning_task.outputs["gcp_resources"], region=REGION
    )

    best_hyperparameters_task = (
        hyperparameter_tuning_job.GetBestHyperparametersOp(
            trials=trials_task.output, study_spec_metrics=metric_spec
        )
    )

    # Construct new worker_pool_specs and
    # train new model based on best hyperparameters
    worker_pool_specs_task = hyperparameter_tuning_job.GetWorkerPoolSpecsOp(
        best_hyperparameters=best_hyperparameters_task.output,
        worker_pool_specs=[
            {
                "machine_spec": {"machine_type": "n1-standard-16"},
                "replica_count": 1,
                "container_spec": {
                    "image_uri": TRAINING_CONTAINER_IMAGE_URI,
                    "args": ["--is_tune=False"],
                },
            }
        ],
    )

    training_task = CustomTrainingJobOp(
        project=PROJECT_ID,
        location=REGION,
        display_name=f"{PIPELINE_NAME}-kfp-training-job",
        worker_pool_specs=worker_pool_specs_task.output,
        base_output_directory=BASE_OUTPUT_DIR,
    )

    model_upload_task = ModelUploadOp(
        project=PROJECT_ID,
        location=REGION,
        display_name=f"{PIPELINE_NAME}-kfp-model-upload-job",
        artifact_uri=f"{BASE_OUTPUT_DIR}/model",
        serving_container_image_uri=SERVING_CONTAINER_IMAGE_URI,
    )
    model_upload_task.after(training_task)

    endpoint_create_task = EndpointCreateOp(
        project=PROJECT_ID,
        location=REGION,
        display_name=f"{PIPELINE_NAME}-kfp-create-endpoint-job",
    )
    endpoint_create_task.after(model_upload_task)

    model_deploy_op = ModelDeployOp(  # pylint: disable=unused-variable
        model=model_upload_task.outputs["model"],
        endpoint=endpoint_create_task.outputs["endpoint"],
        deployed_model_display_name=MODEL_DISPLAY_NAME,
        dedicated_resources_machine_type=SERVING_MACHINE_TYPE,
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1,
        location=REGION,,
    )

Overwriting ./pipeline/pipeline_prebuilt.py


### Compile the pipeline

Let stat by defining the environment variables that will be passed to the pipeline compiler:

In [99]:
#ARTIFACT_STORE = f"gs://{PROJECT_ID}-beer-kfp-artifact-store"
ARTIFACT_STORE = f"gs://{PROJECT_ID}-beer-artifact-store"
PIPELINE_ROOT = f"{ARTIFACT_STORE}/pipeline"
DATA_ROOT = f"{ARTIFACT_STORE}/data"

TRAINING_FILE_PATH = f"{DATA_ROOT}/train.parquet"
VALIDATION_FILE_PATH = f"{DATA_ROOT}/valid.parquet"

%env PIPELINE_ROOT={PIPELINE_ROOT}
%env PROJECT_ID={PROJECT_ID}
%env REGION={REGION}
%env SERVING_CONTAINER_IMAGE_URI={SERVING_CONTAINER_IMAGE_URI}
%env TRAINING_CONTAINER_IMAGE_URI={TRAINING_CONTAINER_IMAGE_URI}
%env TRAINING_FILE_PATH={TRAINING_FILE_PATH}
%env VALIDATION_FILE_PATH={VALIDATION_FILE_PATH}

env: PIPELINE_ROOT=gs://qwiklabs-asl-04-5e165f533cac-beer-artifact-store/pipeline
env: PROJECT_ID=qwiklabs-asl-04-5e165f533cac
env: REGION=us-east1
env: SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest
env: TRAINING_CONTAINER_IMAGE_URI=gcr.io/qwiklabs-asl-04-5e165f533cac/trainer_image_beer_vertex:latest
env: TRAINING_FILE_PATH=gs://qwiklabs-asl-04-5e165f533cac-beer-artifact-store/data/train.parquet
env: VALIDATION_FILE_PATH=gs://qwiklabs-asl-04-5e165f533cac-beer-artifact-store/data/valid.parquet


Let us make sure that the `ARTIFACT_STORE` has been created, and let us create it if not:

In [100]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-04-5e165f533cac-beer-artifact-store/


#### Use the CLI compiler to compile the pipeline

We compile the pipeline from the Python file we generated into a JSON description using the following command:

In [101]:
PIPELINE_JSON = "beer_kfp_pipeline.json"

In [102]:
#!pip3 install https://storage.googleapis.com/ml-pipeline/release/1.8.1/kfp.tar.gz --upgrade
#!pip3 install -U google-cloud-pipeline-components
#설치가 안되어서 터미널에서 root로 --user jupyter 권한으로 설치함

In [103]:
#!dsl-compile-v2 --py pipeline/pipeline.py --output $PIPELINE_JSON
!dsl-compile-v2 --py pipeline/pipeline_prebuilt.py --output $PIPELINE_JSON
#그래도 안되네

Traceback (most recent call last):
  File "/opt/conda/bin/dsl-compile-v2", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/main.py", line 180, in main
    use_experimental=args.use_experimental,
  File "/opt/conda/lib/python3.7/site-packages/kfp/v2/compiler/main.py", line 157, in compile_pyfile
    __import__(os.path.splitext(filename)[0])
  File "pipeline/pipeline_prebuilt.py", line 156
    location=REGION,,
                    ^
SyntaxError: invalid syntax


In [104]:
!head {PIPELINE_JSON}

{
  "pipelineSpec": {
    "components": {
      "comp-custom-training-job": {
        "executorLabel": "exec-custom-training-job",
        "inputDefinitions": {
          "parameters": {
            "base_output_directory": {
              "type": "STRING"
            },


In [105]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)
pipeline = aiplatform.PipelineJob(
    display_name='beer_kfp_pipeline_test',
    template_path=PIPELINE_JSON, 
    enable_caching=False,
)

In [106]:
pipeline.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/547029906128/locations/us-east1/pipelineJobs/beer-kfp-pipeline-20220610034504
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/547029906128/locations/us-east1/pipelineJobs/beer-kfp-pipeline-20220610034504')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-east1/pipelines/runs/beer-kfp-pipeline-20220610034504?project=547029906128
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/547029906128/locations/us-east1/pipelineJobs/beer-kfp-pipeline-20220610034504 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/547029906128/locations/us-east1/pipelineJobs/beer-kfp-pip